# IMPORTS

In [15]:
import pandas as pd

import my_pickle as mp
import my_split as ms
import my_dataframes as md

from importlib import reload

import warnings
warnings.filterwarnings('ignore')

In [23]:
reload(mp)

<module 'my_pickle' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_pickle.py'>

# READ IN

In [17]:
# read in message data
message_df = md.get_message_df()
convo_df = md.get_conversation_df(message_df)
last_df = md.get_lastmessage_df()
user_df = md.get_user_data()

created message dataframe
created conversation dataframe with 0 known errors
created last message dataframe
created user dataframe


# ECT SPLIT

In [18]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
print("split on:         {}".format(pd.to_datetime(cutoff*1000000)))

# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
print("most recent data: {}".format(pd.to_datetime(safe_convo_df.timestamp.max()*1000000)))

split on:         2017-10-02 01:35:58.644000
most recent data: 2017-10-02 01:35:58.644000


# REMOVE ROWS FROM CONVOS WITH MISSING UIDS

In [20]:
safe_convo_df = md.remove_bad_uids(safe_convo_df, user_df)  

0 rows dropped


# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [24]:
mp.json_it(user_df,'data_user')
# mp.pickle_it(message_df,'message_df')  NOT ECT SAFE!
mp.json_it(safe_convo_df,'data_convo')  # make sure pickling safe stuff
# pickle_it(lastmessage_df,'lastmessage_df') NOT ECT SAFE!

# SCRATCH

In [27]:
def my_to_datetime(x):
    if isinstance(x, dict):
        try:
            return pd.to_datetime(x['$date'])
        except:
            return None
    else:
        return None

In [25]:
def get_user_data():

    # read in json as dataframe
    filename = "/Users/gandalf/Documents/data/data_users.json"
    df = pd.read_json(filename)


    df = df.rename(index=str, columns={"_created_at": "created",
                                       "_updated_at": "updated",
                                       "_p_room"    : "has_room",
                                       "_id"        : "uid"})
    df = df.set_index('uid')

    # change dates from strings to date times
    df.created = df.created.apply(lambda x: my_to_datetime(x))
    df.updated = df.updated.apply(lambda x: my_to_datetime(x))
    df.activeAt = df.activeAt.apply(lambda x: my_to_datetime(x))
    df.available = df.available.apply(lambda x: my_to_datetime(x))
    df.birthday = df.birthday.apply(lambda x: my_to_datetime(x))
    df['age'] = 2018-df['birthday'].apply(lambda x: x.year)

#     drop unused columns (dealt with implicitly when rearrange columns
    # col_to_drop = ['_acl','_auth_data_facebook','_hashed_password','_rperm','_wperm','blocked','email','emailVerified','firstName',
    #                   'foundRoommate','groupChat','hometown','hometownCounty','likes','lastName','positions','recommended','username']
    # df = df.drop(col_to_drop, axis = 1)

    # fill in na values
    df = df.fillna({'about':''})


    # create new features
    df['len_about'] = df.about.apply(lambda x: len(x))
    df['has_about'] = df.len_about > 0
    df.has_room = df.has_room.apply(lambda x: isinstance(x,str))
    df.facebookId = df.facebookId.apply(lambda x: isinstance(x,str))
    df.linkedinId = df.linkedinId.apply(lambda x: isinstance(x,str))
    df.picture = df.picture.apply(lambda x: isinstance(x,str))


    # reformat df by renaming and moving around columns
    df = df[['created', 'updated', 'activeAt', 'available', # dates
             'about', 'has_about', 'len_about',            # about
             'birthday', 'age', 'gender', 'location', 'work',      # demographic
             'hometownCity', 'hometownCountry', 'hometownState',     # more demographic
             'college', 'facebookId','linkedinId', 'picture',        # engagement
             'maxCost', 'minCost', 'neighborhoods', 'numRoommates', 'term', 'type', 'has_room',  # room basics
             'amenities', 'hobbies',                                              # room not boolean
             'inRelationship', 'isClean', 'isNight', 'isStudent', 'petsOk', 'smokingOk',   # room boolean
             'onboarded',]]

    # df = df.join(response_df)
    #
    # average_attractiveness = response_df[response_df.attractiveness > 0].mean()['attractiveness']
    # average_responsiveness = response_df[response_df.responsiveness > 0].mean()['responsiveness']
    #
    # df = df.fillna({'messages_sent' : 0,'responses_received': 0,'attractiveness':average_attractiveness,
    #                           'messages_received': 0,'responses_sent': 0, 'responsiveness':average_responsiveness})

    print("created user dataframe")

    return df

In [28]:
df = get_user_data()

created user dataframe
